In [1]:
import boto3
from AwsInfo import AwsInfo

# カスタムライブラリ

In [4]:
from AwsInfo import AwsInfo

In [3]:
aws_info = AwsInfo()

In [11]:
aws_info.get_instance_info()

{'jupyter': {'id': 'i-0ccc4f9d10e7549a7',
  'status': 'running',
  'type': 't2.micro'},
 'jupyterSpot': {'id': 'i-069e83d567631346e',
  'status': 'running',
  'type': 'p2.xlarge'},
 'jupyter_gpu': {'id': 'i-0f95d087c163be336',
  'status': 'stopped',
  'type': 'p2.xlarge'},
 'web': {'id': 'i-0e22faee3f2ebd2ac', 'status': 'running', 'type': 't2.micro'}}

In [33]:
instance_id = aws_info.instance_info["jupyter"]["id"]

# EC2

In [57]:
ec2 = boto3.client('ec2')

In [58]:
response = ec2.describe_instances()

In [59]:
response

{'Reservations': [{'Groups': [],
   'Instances': [{'AmiLaunchIndex': 0,
     'Architecture': 'x86_64',
     'BlockDeviceMappings': [],
     'ClientToken': '',
     'EbsOptimized': False,
     'EnaSupport': True,
     'Hypervisor': 'xen',
     'ImageId': 'ami-6dca2012',
     'InstanceId': 'i-0f95d087c163be336',
     'InstanceType': 'p2.xlarge',
     'KeyName': 'stock-tokyo',
     'LaunchTime': datetime.datetime(2018, 5, 27, 9, 11, 2, tzinfo=tzutc()),
     'Monitoring': {'State': 'disabled'},
     'NetworkInterfaces': [{'Attachment': {'AttachTime': datetime.datetime(2018, 5, 26, 10, 3, 41, tzinfo=tzutc()),
        'AttachmentId': 'eni-attach-3d4dc2d0',
        'DeleteOnTermination': True,
        'DeviceIndex': 0,
        'Status': 'attached'},
       'Description': '',
       'Groups': [{'GroupId': 'sg-553c0a2c', 'GroupName': 'neural'}],
       'Ipv6Addresses': [],
       'MacAddress': '06:48:16:87:15:de',
       'NetworkInterfaceId': 'eni-b677b087',
       'OwnerId': '110144130993',
  

In [73]:
response["Reservations"][0]["Instances"][0]

{'AmiLaunchIndex': 0,
 'Architecture': 'x86_64',
 'BlockDeviceMappings': [],
 'ClientToken': '',
 'EbsOptimized': False,
 'EnaSupport': True,
 'Hypervisor': 'xen',
 'ImageId': 'ami-6dca2012',
 'InstanceId': 'i-0f95d087c163be336',
 'InstanceType': 'p2.xlarge',
 'KeyName': 'stock-tokyo',
 'LaunchTime': datetime.datetime(2018, 5, 27, 9, 11, 2, tzinfo=tzutc()),
 'Monitoring': {'State': 'disabled'},
 'NetworkInterfaces': [{'Attachment': {'AttachTime': datetime.datetime(2018, 5, 26, 10, 3, 41, tzinfo=tzutc()),
    'AttachmentId': 'eni-attach-3d4dc2d0',
    'DeleteOnTermination': True,
    'DeviceIndex': 0,
    'Status': 'attached'},
   'Description': '',
   'Groups': [{'GroupId': 'sg-553c0a2c', 'GroupName': 'neural'}],
   'Ipv6Addresses': [],
   'MacAddress': '06:48:16:87:15:de',
   'NetworkInterfaceId': 'eni-b677b087',
   'OwnerId': '110144130993',
   'PrivateDnsName': 'ip-172-31-21-124.ap-northeast-1.compute.internal',
   'PrivateIpAddress': '172.31.21.124',
   'PrivateIpAddresses': [{'Pri

# ロードバランサー

In [11]:
elb = boto3.client("elbv2")

## ターゲットグループにインスタンスを追加

ターゲットグループarn取得

In [28]:
target_name = "jupyterSpot"

In [29]:
target_group_arn = elb.describe_target_groups(Names=[target_name])["TargetGroups"][0]["TargetGroupArn"]

インスタンスをターゲットグループに登録

In [36]:
elb.register_targets(TargetGroupArn=target_group_arn, Targets=[{
    'Id' : instance_id, 'Port': 8888
}])

{'ResponseMetadata': {'HTTPHeaders': {'content-length': '253',
   'content-type': 'text/xml',
   'date': 'Sat, 09 Jun 2018 07:53:42 GMT',
   'x-amzn-requestid': '3a8c438a-6bba-11e8-856c-8355f68ab097'},
  'HTTPStatusCode': 200,
  'RequestId': '3a8c438a-6bba-11e8-856c-8355f68ab097',
  'RetryAttempts': 0}}

インスタンスをターゲットグループから登録解除

In [37]:
elb.deregister_targets(TargetGroupArn=target_group_arn, Targets=[{
    'Id': instance_id, 'Port': 8888
}])

{'ResponseMetadata': {'HTTPHeaders': {'content-length': '259',
   'content-type': 'text/xml',
   'date': 'Sat, 09 Jun 2018 07:59:40 GMT',
   'x-amzn-requestid': '0fde190c-6bbb-11e8-b0c5-c5d67dc1453e'},
  'HTTPStatusCode': 200,
  'RequestId': '0fde190c-6bbb-11e8-b0c5-c5d67dc1453e',
  'RetryAttempts': 0}}

## その他

In [30]:
elb.describe_target_health(TargetGroupArn=target_group_arn)

{'ResponseMetadata': {'HTTPHeaders': {'content-length': '331',
   'content-type': 'text/xml',
   'date': 'Sat, 09 Jun 2018 07:04:26 GMT',
   'x-amzn-requestid': '58bfc19c-6bb3-11e8-b812-dff030d75c3c'},
  'HTTPStatusCode': 200,
  'RequestId': '58bfc19c-6bb3-11e8-b812-dff030d75c3c',
  'RetryAttempts': 0},
 'TargetHealthDescriptions': []}

# セキュリティグループ

In [8]:
ec2 = boto3.client("ec2")

In [84]:
ec2.describe_security_groups(GroupNames=["neural"])["SecurityGroups"][0]["GroupId"]

'sg-553c0a2c'

In [79]:
ec2.modify_instance_attribute(InstanceId = AwsInfo().get_instance_info()["jupyter"]["id"], 
                             Groups=['sg-553c0a2c'])

{'ResponseMetadata': {'HTTPHeaders': {'content-type': 'text/xml;charset=UTF-8',
   'date': 'Sun, 17 Jun 2018 12:49:47 GMT',
   'server': 'AmazonEC2',
   'transfer-encoding': 'chunked',
   'vary': 'Accept-Encoding'},
  'HTTPStatusCode': 200,
  'RequestId': '7b09cc28-586f-4e55-9ebb-18e0be0ba851',
  'RetryAttempts': 0}}

In [75]:
AwsInfo().get_instance_info()["jupyter"]["id"]

'i-0ccc4f9d10e7549a7'

In [85]:
def assign_security_group(group_name):
    inst_id = AwsInfo().get_instance_info()["jupyter"]["id"]
    ec2c = boto3.client("ec2")
    group_id = ec2c.describe_security_groups(GroupNames=[group_name])["SecurityGroups"][0]["GroupId"]
    ec2c.modify_instance_attribute(InstanceId = inst_id, Groups=[group_id])

In [89]:
assign_security_group("neural")

# ボリューム

In [13]:
ec2c = boto3.client("ec2")

In [10]:
name = "jupyterNotebook"

## ボリューム情報取得

In [11]:
def get_ebs_volume_information_from_name(name):
    info = {}
    ec2c = boto3.client("ec2")
    volume_info = ec2c.describe_volumes()["Volumes"]
    for vol in volume_info:
        tags = vol["Tags"]
        for pair in tags:
            if pair["Key"] == "Name":
                if pair["Value"] == name:
                    info["name"] = name
                    info["id"] = vol["VolumeId"]
                    return info

In [3]:
get_ebs_volume_information_from_name("jupyterNotebook")

{'id': 'vol-0354e76e4955d2637', 'name': 'jupyterNotebook'}

### 関数を試す

In [4]:
info = AwsInfo()

In [6]:
info.get_ebs_volume_information("jupyterNotebook")

{'id': 'vol-0354e76e4955d2637', 'name': 'jupyterNotebook'}

## ボリュームをアタッチ

In [7]:
ec2r = boto3.resource("ec2")

In [8]:
volume = ec2r.Volume(get_ebs_volume_information_from_name(name)["id"])

In [26]:
inst_id = info.get_instance_info()["jupyter"]["id"]

In [27]:
inst_id

'i-0ccc4f9d10e7549a7'

In [21]:
volume.attach_to_instance(Device="xvdf", InstanceId = inst_id)

{'AttachTime': datetime.datetime(2018, 6, 17, 5, 12, 35, 991000, tzinfo=tzutc()),
 'Device': 'xvdf',
 'InstanceId': 'i-0ccc4f9d10e7549a7',
 'ResponseMetadata': {'HTTPHeaders': {'content-type': 'text/xml;charset=UTF-8',
   'date': 'Sun, 17 Jun 2018 05:12:35 GMT',
   'server': 'AmazonEC2',
   'transfer-encoding': 'chunked',
   'vary': 'Accept-Encoding'},
  'HTTPStatusCode': 200,
  'RequestId': '144f66e9-202e-45ea-ac0d-4cdef174fac7',
  'RetryAttempts': 0},
 'State': 'attaching',
 'VolumeId': 'vol-0354e76e4955d2637'}

In [29]:
volume.detach_from_instance(Device="xvdf", InstanceId = inst_id)

{'AttachTime': datetime.datetime(2018, 6, 17, 5, 12, 35, tzinfo=tzutc()),
 'Device': 'xvdf',
 'InstanceId': 'i-0ccc4f9d10e7549a7',
 'ResponseMetadata': {'HTTPHeaders': {'content-type': 'text/xml;charset=UTF-8',
   'date': 'Sun, 17 Jun 2018 05:21:21 GMT',
   'server': 'AmazonEC2',
   'transfer-encoding': 'chunked',
   'vary': 'Accept-Encoding'},
  'HTTPStatusCode': 200,
  'RequestId': '7b7fbe88-1fe4-4c4b-ba82-91f0d907bed5',
  'RetryAttempts': 0},
 'State': 'detaching',
 'VolumeId': 'vol-0354e76e4955d2637'}

### 関数

In [28]:
inst_name = "jupyter"
inst_id = AwsInfo().get_instance_info()[inst_name]["id"]

In [43]:
def attach_volume(volume_name, device_name="xvdf"):
    volume_id = AwsInfo().get_ebs_volume_information(volume_name)["id"]
    ec2r = boto3.resource("ec2")
    volume = ec2r.Volume(volume_id)
    volume.attach_to_instance(Device=device_name, InstanceId=inst_id)

In [45]:
def detach_volume(volume_name, device_name="xvdf"):
    volume_id = AwsInfo().get_ebs_volume_information(volume_name)["id"]
    ec2r = boto3.resource("ec2")
    volume = ec2r.Volume(volume_id)
    volume.detach_from_instance(Device=device_name, InstanceId=inst_id)

In [47]:
attach_volume("jupyterNotebook")

In [48]:
detach_volume("jupyterNotebook")

### 関数を試す

In [56]:
from AwsResource import AwsResource

In [53]:
res = AwsResource("jupyter")

In [54]:
res.attach_volume("jupyterNotebook")

In [55]:
res.detach_volume("jupyterNotebook")

## その他

In [31]:
volume_info = boto3.client("ec2").describe_volumes()["Volumes"]

In [33]:
volume_info

[{'Attachments': [{'AttachTime': datetime.datetime(2017, 12, 10, 6, 20, 25, tzinfo=tzutc()),
    'DeleteOnTermination': True,
    'Device': '/dev/xvda',
    'InstanceId': 'i-0e22faee3f2ebd2ac',
    'State': 'attached',
    'VolumeId': 'vol-031d621365daeaa5e'}],
  'AvailabilityZone': 'ap-northeast-1a',
  'CreateTime': datetime.datetime(2017, 12, 10, 6, 20, 25, 7000, tzinfo=tzutc()),
  'Encrypted': False,
  'Iops': 100,
  'Size': 8,
  'SnapshotId': 'snap-00be5fdba89d48258',
  'State': 'in-use',
  'Tags': [{'Key': 'Name', 'Value': 'webEBS'}],
  'VolumeId': 'vol-031d621365daeaa5e',
  'VolumeType': 'gp2'},
 {'Attachments': [{'AttachTime': datetime.datetime(2018, 1, 30, 14, 43, 11, tzinfo=tzutc()),
    'DeleteOnTermination': True,
    'Device': '/dev/xvda',
    'InstanceId': 'i-0ccc4f9d10e7549a7',
    'State': 'attached',
    'VolumeId': 'vol-079bab626d55cfa29'}],
  'AvailabilityZone': 'ap-northeast-1a',
  'CreateTime': datetime.datetime(2018, 1, 30, 14, 43, 11, 167000, tzinfo=tzutc()),
  'E

In [26]:
volume_info[0]["Tags"]

[{'Key': 'Name', 'Value': 'webEBS'}]